In [ ]:
from selenium import webdriver
from pyshadow.main import Shadow
from selenium.webdriver.common.keys import Keys
import re
import time
import string

dictionary = "american-english.txt"
letters = set(string.ascii_letters)
noAttempt = 6
lenWord = 5

In [ ]:
wordlist = [word.strip() for word in open(dictionary, "r").readlines()]
tryWords = {
    word.lower()
    for word in wordlist    
    if len(word) == lenWord and set(word) < letters
    }
tryWords

In [ ]:
from collections import Counter
from itertools import chain


iterLetter = Counter(chain.from_iterable(tryWords))

iterLetter


In [ ]:
total = len(tryWords)*lenWord
freqLetter = {character: value / total 
                    for character, value in iterLetter.items()}

freqLetter

In [ ]:
def calculate_word_commonality(word):
    score = 0
    for char in word:
        score += freqLetter[char]
    return score / (lenWord - len(set(word)) + 1)

In [ ]:
import operator

def sort_by_word_commonality(words):
    sort_by = operator.itemgetter(1)
    return sorted(
        [(word, calculate_word_commonality(word)) for word in words],
        key = sort_by,
        reverse=True,
    )

def display_word_table(word_commonalities):
    for (word,freq) in word_commonalities:
        print(f"{word:<10} | {freq:<5.2}")


In [ ]:
def input_word():
    while True:
        word = input("Whad'ya put in> ")
        if len(word) == lenWord and word.lower() in tryWords:
            break
    return word.lower()

def input_response():
    print("Put the following based on the response from Wordle:")
    print(" G for Green")
    print(" Y for Yellow")
    print(" ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == lenWord and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [ ]:
def match_word_vector(word, word_vector):
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        if letter not in v_letter:
            return False
    return True

def match(word_vector, possible_words):
    return [word for word in possible_words if match_word_vector(word, word_vector)]

In [ ]:
def enter_word(browser, word):
    shadow = Shadow(browser)
    wordleBot = browser.find_element_by_tag_name('html')
    wordleBot.click()
    time.sleep(1)
    wordleBot.send_keys(word)
    wordleBot.send_keys(Keys.ENTER)
    time.sleep(1)
    finalEvaluation = list(word)
    gameRow = shadow.find_element("game-row[letters="+word+"]")
    for char in set(word):
        gameTile = shadow.find_elements(gameRow,"game-tile[letter=" + char +"]")
        index = -1
        for elem in gameTile:
            if elem.get_attribute("evaluation") == 'present':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'Y' 
            if elem.get_attribute("evaluation") == 'correct':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'G'
            if elem.get_attribute("evaluation") == 'absent':
                finalEvaluation[word.index(char, index + 1, len(word))] = '?'
            index = word.index(char, index + 1, len(word))
    return finalEvaluation

In [47]:
def solve():
    browser = webdriver.Chrome(executable_path ='/usr/local/bin/chromedriver')
    shadow = Shadow(browser)
    browser.get('http://www.powerlanguage.co.uk/wordle/')
    time.sleep(1)
    possible_words = WORDS.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(lenWord)]
    yLetters = set()
    for attempt in range(1, noAttempt + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [ ]:
def solve():
    browser = webdriver.Chrome(executable_path ='/usr/local/bin/chromedriver')
    shadow = Shadow(browser)
    browser.get('https://wordle.berknation.com/')
    time.sleep(10)
    possible_words = WORDS.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(lenWord)]
    yLetters = set()
    for attempt in range(1, noAttempt + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [48]:
solve()

<ipython-input-47-fa2af271ee1b>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path ='/usr/local/bin/chromedriver')


Attempt 1 with 2315 possible words
alter      | 0.39 
later      | 0.39 
alert      | 0.39 
arose      | 0.39 
irate      | 0.39 
stare      | 0.39 
raise      | 0.38 
arise      | 0.38 
renal      | 0.38 
learn      | 0.38 
saner      | 0.38 
snare      | 0.38 
steal      | 0.37 
least      | 0.37 
slate      | 0.37 


<ipython-input-30-51aad33ca172>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  wordleBot = browser.find_element_by_tag_name('html')


QA--QAQA True
Attempt 2 with 450 possible words
salon      | 0.32 
snail      | 0.31 
shoal      | 0.3  
inlay      | 0.29 
piano      | 0.29 
basil      | 0.29 
mason      | 0.28 
voila      | 0.28 
viola      | 0.28 
salvo      | 0.28 
scaly      | 0.28 
chaos      | 0.28 
scald      | 0.28 
scalp      | 0.28 
solid      | 0.28 
QA--QAQA True
Attempt 3 with 33 possible words
daily      | 0.28 
gaily      | 0.27 
cavil      | 0.26 
laugh      | 0.25 
caulk      | 0.25 
madly      | 0.24 
badly      | 0.24 
magic      | 0.24 
gaudy      | 0.22 
vapid      | 0.22 
wacky      | 0.2  
bawdy      | 0.2  
gawky      | 0.18 
cabal      | 0.15 
mafia      | 0.14 
QA--QAQA True
Attempt 4 with 1 possible words
caulk      | 0.25 
QA--QAQA True
Attempt 5 with 1 possible words
caulk      | 0.25 
QA--QAQA True
Attempt 6 with 1 possible words
caulk      | 0.25 


WebDriverException: Message: chrome not reachable
  (Session info: chrome=98.0.4758.80)
Stacktrace:
#0 0x556efdb7e113 <unknown>
#1 0x556efd64656f <unknown>
#2 0x556efd635184 <unknown>
#3 0x556efd635b89 <unknown>
#4 0x556efd6378e2 <unknown>
#5 0x556efd630076 <unknown>
#6 0x556efd647ad2 <unknown>
#7 0x556efd6acfc1 <unknown>
#8 0x556efd699f53 <unknown>
#9 0x556efd66fbda <unknown>
#10 0x556efd670ca5 <unknown>
#11 0x556efdbaf8dd <unknown>
#12 0x556efdbc8a9b <unknown>
#13 0x556efdbb16b5 <unknown>
#14 0x556efdbc9725 <unknown>
#15 0x556efdba508f <unknown>
#16 0x556efdbe6188 <unknown>
#17 0x556efdbe6308 <unknown>
#18 0x556efdc00a6d <unknown>
#19 0x7ffadc703d80 <unknown>
